In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:

def list_intersection(la,lb):
    a = la.copy()
    b = lb.copy()
    inter = []
    while(len(a)*len(b)):
        x = a[0]
        if x in b:
            inter.append(x)
            a.remove(x)
            b.remove(x)
        else:
            a.remove(x)
    return inter

class metrics():
    def __init__(self,pred_csv,split_char = " "):
        print("HELLO")
        self.split_char = split_char
        self.pred_csv = pd.read_csv(pred_csv)
        print(len(self.pred_csv))
        self.pred_csv.dropna(inplace=True)
        print(len(self.pred_csv))
        self.gold_sents = list(self.pred_csv.target_text.values)
        self.pred_sents = list(self.pred_csv.predicted_text.values)
        self.num_sents = len(self.gold_sents)

    def generic_PRF_jiv(self):  # generic Precision, Recall, F1 score implementation as described in jivnesh code
        precisions = []
        recalls = []
        pm = []
        for i in range(self.num_sents):
            # if(self.pred_sents[i].isnan()):
            #     continue
            gold_st = self.gold_sents[i].strip().split(self.split_char)
            pred_st = self.pred_sents[i].strip().split(self.split_char)
            intersection = list_intersection(gold_st,pred_st)

            p = len(intersection)*1.0/len(pred_st)
            r = len(intersection)*1.0/len(gold_st)
            if(pred_st == gold_st):
                pm.append(1.0)
            else:
                pm.append(0.0)

            precisions.append(p)
            recalls.append(r)

        avg_p = np.mean(precisions)*100.0
        avg_r = np.mean(recalls)*100.0
        avg_pm = np.mean(pm)*100
        f1 = 2*avg_p*avg_r/(avg_p + avg_r)


        return avg_p,avg_r,f1,avg_pm

    def generic_PRF(self):  # generic Precision, Recall, F1 score implementation as described in the paper
        avg_p,avg_r,avg_f = 0.0,0.0,0.0
        for i in tqdm(range(self.num_sents)):
            # if(self.pred_sents[i].isnan()):
            #     continue
            gold_st = self.gold_sents[i].strip().split(" ")
            pred_st = self.pred_sents[i].strip().split(" ")
            N = len(gold_st)
            N_dash = len(pred_st)
            gold_st = np.array(gold_st)
            pred = np.array(pred_st)
            t = len(gold_st) - len(pred)
            if(t>0):
                pred = np.append(pred,["$"]*t)
            elif(t<0):
                gold_st = np.append(gold_st,["$"]*(-t))

            c = sum(pred == gold_st)
            p = c/N_dash
            r = c/N
            if(p==0 and r==0):
                f = 0
            else:
                f = 2*(p*r)/(p+r)
            avg_f+=f
            avg_p+=p
            avg_r+=r

        avg_f/=self.num_sents
        avg_r/=self.num_sents
        avg_p/=self.num_sents

        return avg_r,avg_p,avg_f
        

    def pscho_PRD(self):
        #YET TO DO
        pass
# find -maxdepth 1 -name '*.lat' -exec mv -t /home/vp.shivasan/word_segmentation/TransLIST-H22/H22_lattice_files {} +

In [3]:
with open('/home/vp.shivasan/word_segmentation/TransLIST-H22/V0/gold_standard.txt','r') as f:
    input_lines = f.readlines()

with open('/home/vp.shivasan/word_segmentation/TransLIST-H22/V0/gold_standard.txt','r') as f:
    gold_lines = f.readlines()
# sent_df = pd.read_csv('/home/vp.shivasan/word_segmentation/working_dir/data/10K_test-v1/overall_test_sents.csv')
sent_df = pd.read_csv('/home/vp.shivasan/word_segmentation/working_dir/data/new_training_data/sentence_data/collection1_test_sent.csv')
with open('/home/vp.shivasan/word_segmentation/TransLIST-H22/V0/pred_standard.txt','r') as f:
    pred_lines = f.readlines()

In [4]:
input_lines[0]

'kzuD_boDAya_lavaFgena_nara_mUtreNa_vA_punar_\n'

In [4]:
pro_gold_lines = []
# pro_input_lines_from_char = []
for line in gold_lines:
    pro_gold_lines.append(line.strip("\n").replace("__","_").strip("_"))

pro_input_lines = []
for line in input_lines:
    pro_input_lines.append(line.strip("\n").replace("__","_").strip("_"))
pro_input_lines

['yA_paSuzu_taya_Rte_yajQam',
 'yena_paTA_vEvasvato_yamo_rAjA_no_yayO_agnir_nas_tena_nayatu_prajAnan_vESvAnaraH_paTikRd_viSvak_RzwiH',
 'Kara_uzwra_yAna_hasti_aSva_nO_vRkza_giri_rohaNe',
 'om_jQAnine_Wa_Wa_SiKAyE_namaH',
 'om_sarva_tejaH_aDipataye_Wa_Wa_astrAya_namaH',
 'OdBidam_tu_punar_vanaspati_vAnaspatya_vIruD_OzaDi_Bedena_caturviDam_Bavati',
 'Am_e_taH',
 'ha_e_ti',
 'diva_O_t',
 'na_mAF_yoge',
 'Svas_Svas_BUyAn_Bavati',
 'di<unk>AdiByaH_Syan',
 'pratilome_hi_samyak_OzaDam_avatizWate_anupraviSati_romakUpAn_sveda_vAhiBiH_ca_sirA_muKEH_vIryam_prApnoti',
 "itara_pANi_aFguzWa_antareNa_upavIti_tvAt_dakziNena_vA_savya_upagRhItena_pitar_idaM_te_'rGyaM_pitAmaha_idam_te_'rGyaM_prapitAmaha_idam_te_'rGyam_ity_ap_pUrvam",
 'om_sahasraraSmaye_Wa_Wa_kavacAya_namaH',
 'trizwuB_tvA_avatu_bRhat_sAma_paQcadaSa_stomo_grIzma_RtuH_kzatraM_draviNam',
 'trizwuB_sahasram_eke_jagatI_sahasram_eke_anuzwuB_sahasram_eke',
 'kzuD_yasya_tasya_Bukte_anne_tRptiH_brAhmaNa_jAyate',
 'tad_bahis_waFkaNena_eva_loNamRt

In [9]:
# pro_gold_lines = []
# # pro_input_lines_from_char = []
# # for line in gold_lines:
# #     pro_gold_lines.append(line.strip("\n").replace("$","_").replace("__","_").strip("_"))

# pro_input_lines = []
# for line in input_lines:
#     pro_input_lines.append(line.strip("\n").strip("_"))#.replace("_","").replace("$","_").
# pro_input_lines

['atastvamsvapatimnijaSarIrasyakaticiddinasTAyiyOvanasyapuruzaantararamaNAtgRhANaPalam',
 'daSagosahasradaHsamyakSakreNasahamodate',
 'sRzwamUtrapurIzANiraktapittaharANica',
 'tadsidDatElampawagAlitamvastrapUtamgRhItvAtadanutadpaScAt',
 'avagAhakuwIkarzUprastaraaByaFgabastiBiH',
 'mRtaarTamtadahedAnamtrizusyAtOrDvadehikam',
 'vibuDyetacasasvedaHdAhatRztApapIxitaH',
 'gupUDUpavicCipaNipaniByaHAyaH',
 'aDascaUrDvamcatiryakcajagatApyamivaaBavat',
 'vanotpalasahasreNapUritepuwanaOzaDam',
 'vyaTitasinDumSanEsamaralokavaDUjaGanEHGanEH',
 'vacanamnaraSArdUlavajraAyuDamapisvayam',
 'itiSrIskAndemahApurANeekASItisAhasryAmsaMhitAyAmpaQcameAvantyaKaNxerevAKaNxeSrIpativivAhavarNanamnAmacaturnavatiuttaraSatatamaHaDyAyaH',
 'raTaaSvamhastinamCattramDanamDAnyampaSUnstriyaH',
 'aTaitarampunaretyaEndramsamasTApayat',
 'kalOpramAdakaHrogaHsatatamkzuDBayAnica',
 'daradamhiFgulamhemAhvAcokapatramAtramhiFgulaAdyAHkarzamAtramdantIbIjamtadsamamgodugDenatriguQjampibet',
 'avAkSirAHcaBagavAnudatizWatacandramAH

In [5]:
pro_pred_lines = []
for line in pred_lines:
    pro_pred_lines.append(line.strip("\n").replace("$","_").replace("__","_").strip("_"))
pro_pred_lines

['yA_paSuzu_taya_Rteya_jQam',
 'yena_paTA_vEvasvato_yamaH_rAjA_no_yayO_agnir_nas_tena_nayatu_prajAnan_vESvAnaraH_paTikRd_viSva_gRzwiH',
 'Kara_uzwrayAna_hastya_SvanOuRkza_girirohaNe',
 'xm_jQAnine_WaWa_SiKAyE_namaH',
 'xm_sarva_tejaH_aDipataye_WaWaa_astrAya_namaH',
 'OdBidam_tu_punar_vanaspati_vAnaspatya_vIru_t_a_OzaDi_Bedena_caturviDam_Bavati',
 'Am_etaH',
 'ha_eti',
 'diva_Ot',
 'na_ma_aFyige',
 'Sva_SSvas_BUyAn_Bavati',
 'divAdiByaH_Syan',
 'pratilome_hi_samyan_OzaDam_avatizWate_acu_praviSati_romakUpAn_sveda_vAhiBiH_ca_sirAmuKEH_vIryam_prApnoti',
 'itarmpANi_aFguzWA_antareNa_upavIti_tvAt_dakzoNena_vA_savyopagRhItena_pitar_idam_te_arXyam_pitAmaha_idam_te_aruyam_prmpitAmaha_idam_te_aruyam_itylmppUrvam',
 'xm_sahasraraSmaye_WaWa_kavacAya_namaH',
 'trizwuptvAvatu_bRhatsAma_paQcadaSa_stomaH_grIzmaH_RtuH_kzatram_draviNam',
 'trizwuH_sahasram_eke_jagatI_sahasram_eke_anuzwuH_sahasram_eke',
 'kzudyasya_tasya_Bukte_anne_tRptiH_brAhmaNa_jAyate',
 'tad_bahizwavkaNena_eva_loNamRttikayA_tatas',
 

In [7]:
with open("/home/vp.shivasan/word_segmentation/TransLIST-H22/sktWS/overall_test_cut_150.txt","r") as f:
    test_char_dat = f.readlines()

In [8]:
from thefuzz import process

def get_gold_lines(char_dat,good_inputs):
    gold_lines = []
    input_lines = []
    input_str = ""
    output_str = ""
    for ele in char_dat:
        if(ele == "\n"):
            # item = process.extractOne(input_str, good_inputs)
            # print(item)
            # if(item[1]<95):
            if(input_str not in good_inputs):
                input_str = ""
                output_str = ""
                continue
            input_lines.append(input_str)
            gold_lines.append(output_str)
            input_str = ""
            output_str = ""
            continue
        i,o = ele.strip("\n").split(" ")
        input_str+=i
        output_str+=o
    # assert len(good_inputs) == len(gold_lines)
    return input_lines,gold_lines

        

In [9]:
sent_df

,UID,input,output
0,605304,yad_ezo_'nuzwupSirAH_pragATo_Bavati_virAx_eva_...,yat_ezaH_anuzwuB_SirAH_pragATaH_Bavati_virAj_e...
1,32075,yA_paSuzu_taya_RteyajQam,yA_paSuzu_taya_Rte_yajQam
2,54157,yena_paTA_vEvasvato_yamo_rAjA_no_yayO_agnir_na...,yena_paTA_vEvasvato_yamo_rAjA_no_yayO_agnir_na...
3,18574,hRdayasya_hRlleKayadaNlAsezu,hRdayasya_hRd_leKa_yat_aN_lAsezu
4,449886,KarozwrayAnahastyaSvanOvRkzagirirohaNe,Kara_uzwra_yAna_hasti_aSva_nO_vRkza_giri_rohaNe
...,...,...,...
10752,-1,iva_Aruka_kAka_kaFka_kurara_cAza_BAsa_SaSaGAti...,iva_Aruka_kAka_kaFka_kurara_cAza_BAsa_SaSaGAti...
10753,-1,pittAnnIlaharidrAmlakzAramaQjizWASoNitamehAH_l...,pittAt_nIla_haridrA_amla_kzAra_maQjizWA_SoNita...
10754,-1,arkAlarkakaraQjadvayanAgadantImayUrakaBArgIrAs...,arka_alarka_karaQja_dvaya_nAgadantI_mayUraka_B...
10755,-1,elA_tagara_kuzWa_mAMsI_DyAmaka_tvac_pattra_nAg...,elA_tagara_kuzWa_mAMsI_DyAmaka_tvac_pattra_nAg...


In [10]:
input_lines_from_char,gold_lines = get_gold_lines(test_char_dat,pro_input_lines)
print(len(input_lines_from_char))
# len(gold_lines)
pro_gold_lines = []
pro_input_lines_from_char = []
for line in gold_lines:
    pro_gold_lines.append(line.strip("\n").replace("$","_").replace("__","_").strip("_"))

for line in input_lines_from_char:
    pro_input_lines_from_char.append(line.strip("\n").replace("$","_").replace("__","_").strip("_"))
print(len(pro_gold_lines),len(pro_input_lines_from_char))
# pro_gold_lines

5473
5473 5473


In [11]:
ct = 0
for i,input_line in enumerate( pro_input_lines):
    if(input_line not in pro_input_lines_from_char):
        print(i)
        print(input_line)
        ct+=1
print(ct)

0


In [12]:
pro_gold_lines

['yA_paSuzu_taya_RteyajQam',
 'yena_paTA_vEvasvato_yamo_rAjA_no_yayO_agnir_nas_tena_nayatu_prajAnan_vESvAnaraH_paTikRd_viSvagRzwiH',
 'KarozwrayAnahastyaSvanOvRkzagirirohaNe',
 'oM_jQAnine_WaWa_SiKAyE_namaH',
 "oM_sarvatejo'Dipataye_WaWa_astrAya_namaH",
 'OdBidaM_tu_punarvanaspativAnaspatyavIrudOzaDiBedena_caturviDaM_Bavati',
 'Am_etaH',
 'ha_eti',
 'diva_Ot',
 'na_mAFyoge',
 'SvaSSvas_BUyAn_Bavati',
 'divAdiByaH_Syan',
 "pratilome_hi_samyagOzaDamavatizWate_'nupraviSati_romakUpAn_svedavAhiBiS_ca_sirAmuKEr_vIryaM_prApnoti",
 "itarapANyaFguzWAntareNopavItitvAt_dakziNena_vA_savyopagRhItena_pitar_idaM_te_'rGyaM_pitAmahedaM_te_'rGyaM_prapitAmahedaM_te_'rGyam_ity_appUrvam",
 'oM_sahasraraSmaye_WaWa_kavacAya_namaH',
 'trizwuptvAvatu_bRhatsAma_paQcadaSa_stomo_grIzma_RtuH_kzatraM_draviNam',
 'trizwupsahasram_eke_jagatIsahasram_eke_anuzwupsahasram_eke',
 "kzudyasya_tasya_Bukte_'nne_tRptirbrAhmaNa_jAyate",
 'tadbahizwaFkaNenEva_loNamRttikayA_tataH',
 'agnir_nas_tena_nayatu_prajAnan_vESvAnaraH_paT

In [13]:
pro_input_lines_from_char

['yA_paSuzu_taya_RteyajQam',
 'yena_paTA_vEvasvato_yamo_rAjA_no_yayO_agnir_nas_tena_nayatu_prajAnan_vESvAnaraH_paTikRd_viSvagRzwiH',
 'KarozwrayAnahastyaSvanOvRkzagirirohaNe',
 'oM_jQAnine_WaWa_SiKAyE_namaH',
 "oM_sarvatejo'Dipataye_WaWa_astrAya_namaH",
 'OdBidaM_tu_punarvanaspativAnaspatyavIrudOzaDiBedena_caturviDaM_Bavati',
 'Am_etaH',
 'ha_eti',
 'diva_Ot',
 'na_mAFyoge',
 'SvaSSvas_BUyAn_Bavati',
 'divAdiByaH_Syan',
 "pratilome_hi_samyagOzaDamavatizWate_'nupraviSati_romakUpAn_svedavAhiBiS_ca_sirAmuKEr_vIryaM_prApnoti",
 "itarapANyaFguzWAntareNopavItitvAt_dakziNena_vA_savyopagRhItena_pitar_idaM_te_'rGyaM_pitAmahedaM_te_'rGyaM_prapitAmahedaM_te_'rGyam_ity_appUrvam",
 'oM_sahasraraSmaye_WaWa_kavacAya_namaH',
 'trizwuptvAvatu_bRhatsAma_paQcadaSa_stomo_grIzma_RtuH_kzatraM_draviNam',
 'trizwupsahasram_eke_jagatIsahasram_eke_anuzwupsahasram_eke',
 "kzudyasya_tasya_Bukte_'nne_tRptirbrAhmaNa_jAyate",
 'tadbahizwaFkaNenEva_loNamRttikayA_tataH',
 'agnir_nas_tena_nayatu_prajAnan_vESvAnaraH_paT

In [14]:
len(pro_input_lines)

5473

In [15]:
len(pro_gold_lines),len(pro_pred_lines)

(5473, 5473)

In [6]:
predict_df = pd.DataFrame(columns=['target_text','predicted_text'])
# predict_df['input_text'] = pro_input_lines
predict_df['target_text'] = pro_gold_lines
predict_df['predicted_text'] = pro_pred_lines
predict_df

,target_text,predicted_text
0,yA_paSuzu_taya_Rte_yajQam,yA_paSuzu_taya_Rteya_jQam
1,yena_paTA_vEvasvato_yamo_rAjA_no_yayO_agnir_na...,yena_paTA_vEvasvato_yamaH_rAjA_no_yayO_agnir_n...
2,Kara_uzwra_yAna_hasti_aSva_nO_vRkza_giri_rohaNe,Kara_uzwrayAna_hastya_SvanOuRkza_girirohaNe
3,om_jQAnine_Wa_Wa_SiKAyE_namaH,xm_jQAnine_WaWa_SiKAyE_namaH
4,om_sarva_tejaH_aDipataye_Wa_Wa_astrAya_namaH,xm_sarva_tejaH_aDipataye_WaWaa_astrAya_namaH
...,...,...
119,pakva_Saya_udBavaH_api_evam_vAyuH_tIvra_ruj_AS...,pakva_Saya_udBavaH_api_evam_vAyuH_tIvraruja_AS...
120,tAni_ca_eva_tu_mAnAni_azwO_zax_vA_caturTakam,tAni_ca_eva_tu_mAnAni_azwO_zaa_vA_caturTakam
121,hema_alaMkAriNaH_kAryAH_paQcaviMSati_RtvijaH,hemA_aMkAriNaH_kAryAH_paQcaviMSati_RtvijaH
122,a_taH_ini_WanO,atas_iniWanO


In [7]:
predict_df.to_csv('/home/vp.shivasan/word_segmentation/TransLIST-H22/V0/metric-files/collection3_test_met.csv',index=None)

In [9]:
# from helpers import metrics
tgt_filename = '/home/vp.shivasan/word_segmentation/TransLIST-H22/V0/metric-files/overall_test_met.csv'
custom_metrics = metrics(tgt_filename,split_char="_")
P,R,F,PM = custom_metrics.generic_PRF_jiv()
print("Precision: {}\n Recall: {}\n F1: {}\n PM:{}".format(P,R,F,PM))

HELLO
5472
5472
Precision: 43.330366574679196
 Recall: 40.07570530815479
 F1: 41.63953444973486
 PM:0.7309941520467835


In [66]:
predict_df

,target_text,predicted_text
0,yA_paSuzu_taya_RteyajQam,yA_paSuzu_teya_RtA_iyajQam
1,yena_paTA_vEvasvato_yamo_rAjA_no_yayO_agnir_na...,yena_paTA_vEvasvata_yam_rAjA_n_ye_O_at_nip_nas...
2,KarozwrayAnahastyaSvanOvRkzagirirohaNe,Kara_uzwreyAna_haetya_Svani_uRkzam_iri_rMhaNe
3,oM_jQAnine_WaWa_SiKAyE_namaH,xm_jQAnine_WaWa_SiKAyE_namaH
4,oM_sarvatejo'Dipataye_WaWa_astrAya_namaH,xm_sarvatejaH_aDipateya_WaWa_aetrAya_namaH
...,...,...
5468,saMyogAt_ca_ekAnta_hitAni_ekAnta_ahitAni_yasmA...,saMyogAt_ca_ekAnta_hitAni_ekAnta_ahitAnk_yasmA...
5469,SakRdvamettamAmaM_svedana_pAcanEH_ca_aTetaraM_...,SakRt_vamsttamAmam_svedana_pAcanEH_ca_aTstaram...
5470,go_aSva_avi_aja_iBa_eNa_Kara_uzwra_jAtEH_SakRd...,b_taSva_aiktajat_iBa_eNa_Kara_uzwra_rAtEH_SakR...
5471,sahasra_AyUH_naraH_vItapApmA_SrutaDaraH_Bavet,sahasra_yAUH_naraH_vItmpApmA_SrutaDaraH_Bavst


In [67]:
sent_df

,UID,input,output
0,605304,yad_ezo_'nuzwupSirAH_pragATo_Bavati_virAx_eva_...,yat_ezaH_anuzwuB_SirAH_pragATaH_Bavati_virAj_e...
1,32075,yA_paSuzu_taya_RteyajQam,yA_paSuzu_taya_Rte_yajQam
2,54157,yena_paTA_vEvasvato_yamo_rAjA_no_yayO_agnir_na...,yena_paTA_vEvasvato_yamo_rAjA_no_yayO_agnir_na...
3,18574,hRdayasya_hRlleKayadaNlAsezu,hRdayasya_hRd_leKa_yat_aN_lAsezu
4,449886,KarozwrayAnahastyaSvanOvRkzagirirohaNe,Kara_uzwra_yAna_hasti_aSva_nO_vRkza_giri_rohaNe
...,...,...,...
10752,-1,iva_Aruka_kAka_kaFka_kurara_cAza_BAsa_SaSaGAti...,iva_Aruka_kAka_kaFka_kurara_cAza_BAsa_SaSaGAti...
10753,-1,pittAnnIlaharidrAmlakzAramaQjizWASoNitamehAH_l...,pittAt_nIla_haridrA_amla_kzAra_maQjizWA_SoNita...
10754,-1,arkAlarkakaraQjadvayanAgadantImayUrakaBArgIrAs...,arka_alarka_karaQja_dvaya_nAgadantI_mayUraka_B...
10755,-1,elA_tagara_kuzWa_mAMsI_DyAmaka_tvac_pattra_nAg...,elA_tagara_kuzWa_mAMsI_DyAmaka_tvac_pattra_nAg...


In [70]:
temp_df = pd.merge(predict_df,sent_df,left_on="target_text",right_on="input")
temp_df

,target_text,predicted_text,UID,input,output
0,yA_paSuzu_taya_RteyajQam,yA_paSuzu_teya_RtA_iyajQam,32075,yA_paSuzu_taya_RteyajQam,yA_paSuzu_taya_Rte_yajQam
1,yena_paTA_vEvasvato_yamo_rAjA_no_yayO_agnir_na...,yena_paTA_vEvasvata_yam_rAjA_n_ye_O_at_nip_nas...,54157,yena_paTA_vEvasvato_yamo_rAjA_no_yayO_agnir_na...,yena_paTA_vEvasvato_yamo_rAjA_no_yayO_agnir_na...
2,KarozwrayAnahastyaSvanOvRkzagirirohaNe,Kara_uzwreyAna_haetya_Svani_uRkzam_iri_rMhaNe,449886,KarozwrayAnahastyaSvanOvRkzagirirohaNe,Kara_uzwra_yAna_hasti_aSva_nO_vRkza_giri_rohaNe
3,oM_jQAnine_WaWa_SiKAyE_namaH,xm_jQAnine_WaWa_SiKAyE_namaH,441830,oM_jQAnine_WaWa_SiKAyE_namaH,om_jQAnine_Wa_Wa_SiKAyE_namaH
4,oM_sarvatejo'Dipataye_WaWa_astrAya_namaH,xm_sarvatejaH_aDipateya_WaWa_aetrAya_namaH,441828,oM_sarvatejo'Dipataye_WaWa_astrAya_namaH,om_sarva_tejaH_aDipataye_Wa_Wa_astrAya_namaH
...,...,...,...,...,...
5470,saMyogAt_ca_ekAnta_hitAni_ekAnta_ahitAni_yasmA...,saMyogAt_ca_ekAnta_hitAni_ekAnta_ahitAnk_yasmA...,-1,saMyogAt_ca_ekAnta_hitAni_ekAnta_ahitAni_yasmA...,saMyogAt_ca_ekAnta_hitAni_ekAnta_ahitAni_yasmA...
5471,SakRdvamettamAmaM_svedana_pAcanEH_ca_aTetaraM_...,SakRt_vamsttamAmam_svedana_pAcanEH_ca_aTstaram...,-1,SakRdvamettamAmaM_svedana_pAcanEH_ca_aTetaraM_...,SakRt_vamet_tam_Amam_svedana_pAcanEH_ca_aTa_it...
5472,go_aSva_avi_aja_iBa_eNa_Kara_uzwra_jAtEH_SakRd...,b_taSva_aiktajat_iBa_eNa_Kara_uzwra_rAtEH_SakR...,-1,go_aSva_avi_aja_iBa_eNa_Kara_uzwra_jAtEH_SakRd...,go_aSva_avi_aja_iBa_eNa_Kara_uzwra_jAtEH_SakRt...
5473,sahasra_AyUH_naraH_vItapApmA_SrutaDaraH_Bavet,sahasra_yAUH_naraH_vItmpApmA_SrutaDaraH_Bavst,-1,sahasra_AyUH_naraH_vItapApmA_SrutaDaraH_Bavet,sahasra_AyUH_naraH_vIta_pApmA_Sruta_DaraH_Bavet


In [82]:
droped_df = temp_df.drop_duplicates(keep="first")
droped_df

,target_text,predicted_text,UID,input,output
0,yA_paSuzu_taya_RteyajQam,yA_paSuzu_teya_RtA_iyajQam,32075,yA_paSuzu_taya_RteyajQam,yA_paSuzu_taya_Rte_yajQam
1,yena_paTA_vEvasvato_yamo_rAjA_no_yayO_agnir_na...,yena_paTA_vEvasvata_yam_rAjA_n_ye_O_at_nip_nas...,54157,yena_paTA_vEvasvato_yamo_rAjA_no_yayO_agnir_na...,yena_paTA_vEvasvato_yamo_rAjA_no_yayO_agnir_na...
2,KarozwrayAnahastyaSvanOvRkzagirirohaNe,Kara_uzwreyAna_haetya_Svani_uRkzam_iri_rMhaNe,449886,KarozwrayAnahastyaSvanOvRkzagirirohaNe,Kara_uzwra_yAna_hasti_aSva_nO_vRkza_giri_rohaNe
3,oM_jQAnine_WaWa_SiKAyE_namaH,xm_jQAnine_WaWa_SiKAyE_namaH,441830,oM_jQAnine_WaWa_SiKAyE_namaH,om_jQAnine_Wa_Wa_SiKAyE_namaH
4,oM_sarvatejo'Dipataye_WaWa_astrAya_namaH,xm_sarvatejaH_aDipateya_WaWa_aetrAya_namaH,441828,oM_sarvatejo'Dipataye_WaWa_astrAya_namaH,om_sarva_tejaH_aDipataye_Wa_Wa_astrAya_namaH
...,...,...,...,...,...
5470,saMyogAt_ca_ekAnta_hitAni_ekAnta_ahitAni_yasmA...,saMyogAt_ca_ekAnta_hitAni_ekAnta_ahitAnk_yasmA...,-1,saMyogAt_ca_ekAnta_hitAni_ekAnta_ahitAni_yasmA...,saMyogAt_ca_ekAnta_hitAni_ekAnta_ahitAni_yasmA...
5471,SakRdvamettamAmaM_svedana_pAcanEH_ca_aTetaraM_...,SakRt_vamsttamAmam_svedana_pAcanEH_ca_aTstaram...,-1,SakRdvamettamAmaM_svedana_pAcanEH_ca_aTetaraM_...,SakRt_vamet_tam_Amam_svedana_pAcanEH_ca_aTa_it...
5472,go_aSva_avi_aja_iBa_eNa_Kara_uzwra_jAtEH_SakRd...,b_taSva_aiktajat_iBa_eNa_Kara_uzwra_rAtEH_SakR...,-1,go_aSva_avi_aja_iBa_eNa_Kara_uzwra_jAtEH_SakRd...,go_aSva_avi_aja_iBa_eNa_Kara_uzwra_jAtEH_SakRt...
5473,sahasra_AyUH_naraH_vItapApmA_SrutaDaraH_Bavet,sahasra_yAUH_naraH_vItmpApmA_SrutaDaraH_Bavst,-1,sahasra_AyUH_naraH_vItapApmA_SrutaDaraH_Bavet,sahasra_AyUH_naraH_vIta_pApmA_Sruta_DaraH_Bavet


In [88]:
droped_df[["output","predicted_text"]].rename(columns={"output":"target_text"}).to_csv('/home/vp.shivasan/word_segmentation/TransLIST-H22/V0/metric-files/overall_test_met.csv',index=None)